In [1]:
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import json
import numpy as np
from config import OUTPUT_PATH
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [4]:
with open("../" + OUTPUT_PATH + f"사업계획서자손2.json", "r", encoding="UTF-8") as fp:
    text= fp.read()
    
json_data = json.loads(text)

splitter = RecursiveJsonSplitter(max_chunk_size=400)
docs = splitter.create_documents(texts=[json_data], ensure_ascii=False)
#docs = splitter.split_json(json_data=json_data)

In [5]:
len(docs)

613

In [6]:
docs[40:50]

[Document(page_content='{"자본금_0": {"Year": "2023", "Value": {"Samsung": 897514, "Hyundai": "-", "Kepco": "-", "Sum": 897514, "Avg": 897514.0}, "Ancestors": ["자본", "지배기업 소유주지분"]}}'),
 Document(page_content='{"보통주자본금_0": {"Year": "2023", "Value": {"Samsung": 778047, "Hyundai": "-", "Kepco": "-", "Sum": 778047, "Avg": 778047.0}, "Ancestors": ["자본", "지배기업 소유주지분", "자본금"]}}'),
 Document(page_content='{"우선주자본금_0": {"Year": "2023", "Value": {"Samsung": 119467, "Hyundai": "-", "Kepco": "-", "Sum": 119467, "Avg": 119467.0}, "Ancestors": ["자본", "지배기업 소유주지분", "자본금"]}}'),
 Document(page_content='{"주식발행초과금_0": {"Year": "2023", "Value": {"Samsung": 4403893, "Hyundai": "-", "Kepco": "-", "Sum": 4403893, "Avg": 4403893.0}, "Ancestors": ["자본", "지배기업 소유주지분"]}}'),
 Document(page_content='{"지배기업소유주지분_0": {"Year": "2023", "Value": {"Samsung": "-", "Hyundai": 92497311, "Kepco": "-", "Sum": 92497311, "Avg": 92497311.0}, "Ancestors": ["자본"]}}'),
 Document(page_content='{"기타자본항목_1": {"Year": "2023", "Value": {"

In [16]:
type(docs[40])

langchain_core.documents.base.Document

In [61]:
max = 0
max_doc = ""
for doc in docs:
    if max < len(doc.page_content):
        max = len(doc.page_content)
        max_doc = doc
print(max)
print(max_doc)

169
page_content='{"부채총계_2": {"Year": "2021", "Value": {"Samsung": 121721227, "Hyundai": 151330626, "Kepco": 145797021, "Sum": 418848874, "Avg": 139616291.33333334}, "Ancestors": ["부채"]}}'


In [5]:
doc_texts = []
for idx in range(len(docs)):
    doc_texts.append(docs[idx].page_content)

In [26]:
doc_texts[40]

NameError: name 'doc_texts' is not defined

In [4]:

text_embedding_model = OpenAIEmbeddings()

#embedded_texts = text_embedding_model.embed_documents(doc_texts)


In [5]:

db = Chroma.from_documents(
    docs,
    text_embedding_model,
    collection_name = "saup",
    persist_directory = "../" + OUTPUT_PATH + "chromadb",
    collection_metadata = {"hnsw:space":"cosine"}       # 이거 뭘까 공부할 필요가 있음
)

In [2]:
query = "비지배지분"
retriever = db.as_retriever(
    search_type = "similarity",
    search_kwargs={"k":20}
)
docs = retriever.invoke(query)
for doc in docs:
    print(doc.page_content)

NameError: name 'db' is not defined

In [56]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)